In [1]:
# Importació de mòduls

import pandas as pd
import tensorflow as tf

In [ ]:
# Accés a Google Drive

from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Importació de dades

# Google Drive
df_train = pd.read_csv('/content/drive/My Drive/Kaggle/train_sales.csv')
df_eval = pd.read_csv('/content/drive/My Drive/Kaggle/train_eval.csv')

# Local
#df_train = pd.read_csv('train_sales.csv')
#df_eval = pd.read_csv('train_eval.csv')

y_train = df_train.pop('item_cnt').clip(0,20)
y_eval = df_eval.pop('item_cnt').clip(0,20)

In [46]:
# Funció que defineix els atributs a TF

NUMERIC_COLUMNS = ['date_block_num','month_days','itemshop_id_freq','item_id_freq','shop_id_freq',
                   'item_category_id_freq','cat_name1_freq','cat_name2_freq']

feature_columns = []

for feature_name in NUMERIC_COLUMNS:
      feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype=tf.float32))
        

In [47]:
# Funcions d'entrada de dades

NUM_EXAMPLES = len(y_train)

def make_input_fn(X, y, n_epochs=None, shuffle=True):
    def input_fn():
        dataset = tf.data.Dataset.from_tensor_slices((dict(X), y))
        if shuffle:
            dataset = dataset.shuffle(NUM_EXAMPLES)

        dataset = dataset.repeat(n_epochs)
        dataset = dataset.batch(NUM_EXAMPLES)
        
        return dataset

    return input_fn

train_input_fn = make_input_fn(df_train, y_train)
eval_input_fn = make_input_fn(df_eval, y_eval, shuffle=False, n_epochs=1)

In [ ]:
est = tf.estimator.BoostedTreesRegressor(feature_columns, n_batches_per_layer=1)

# Train model.
est.train(train_input_fn, max_steps=100)

# Evaluation.
result = est.evaluate(eval_input_fn)
print(pd.Series(result))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './trained_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
I

In [6]:
# RMSE del conjunt de validació

y_pred = model.predict(X_eval)
np.sqrt(mean_squared_error(y_pred.clip(0,20), y_eval))

0.9250438851917503

## Predicció

In [11]:
# Dades

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

X = train.drop('item_cnt', 1)
y = train['item_cnt'].clip(0,20)

In [12]:
# Entrenament

model = XGBRegressor(max_depth=12, n_estimators= 25, objective='reg:squarederror')\
                    .fit(X, y, eval_metric="rmse", eval_set=[(X,y)])

[0]	validation_0-rmse:0.998572
[1]	validation_0-rmse:0.937765
[2]	validation_0-rmse:0.880901
[3]	validation_0-rmse:0.835832
[4]	validation_0-rmse:0.792022
[5]	validation_0-rmse:0.75706
[6]	validation_0-rmse:0.729182
[7]	validation_0-rmse:0.697069
[8]	validation_0-rmse:0.674709
[9]	validation_0-rmse:0.65435
[10]	validation_0-rmse:0.638751
[11]	validation_0-rmse:0.622675
[12]	validation_0-rmse:0.609623
[13]	validation_0-rmse:0.591777
[14]	validation_0-rmse:0.581575
[15]	validation_0-rmse:0.572783
[16]	validation_0-rmse:0.561887
[17]	validation_0-rmse:0.55426
[18]	validation_0-rmse:0.546673
[19]	validation_0-rmse:0.539405
[20]	validation_0-rmse:0.533767
[21]	validation_0-rmse:0.528384
[22]	validation_0-rmse:0.523632
[23]	validation_0-rmse:0.518914
[24]	validation_0-rmse:0.514892


In [13]:
# Predicció

y_pred = model.predict(test[X.columns]).clip(0,20)
submission = pd.DataFrame({'ID':test['ID'], 'item_cnt_month': y_pred})
submission.to_csv('submission.csv', index = False)

In [14]:
# RMSE al conjunt d'entrenament

np.sqrt(mean_squared_error(model.predict(train[X.columns]), y))

0.5156962412681523

RMSE al conjunt test (public score): 1.06197